<a href="https://colab.research.google.com/github/aurotripathy/raggedy-ann/blob/main/llama_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers
!pip install torch --index-url https://download.pytorch.org/whl/cu121/
!python -m pip install -q huggingface_hub
!pip install accelerate>=0.26.0
!pip install -U scikit-learn scipy matplotlib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://download.pytorch.org/whl/cu121/, https://pypi.ngc.nvidia.com


In [2]:
# go here for token: https://huggingface.co/settings/tokens
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import torch
print(torch.__version__)

2.4.1+cu121


In [5]:
%pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.


In [6]:
import transformers
import torch
model_id = "meta-llama/Meta-Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="cuda:0"
)

pipeline("Hey how are you doing today?")

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/opt/vast-jupyter/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


[{'generated_text': 'Hey how are you doing today? I am doing great, and I hope you are as well'}]

In [7]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="cuda")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [57]:
import numpy as np
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt")
    inputs = inputs.to("cuda")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state
    embeddings = embeddings.to("cpu").float()
    print(embeddings)
    print(f'{embeddings.size()}')
    return embeddings.mean(dim=0).squeeze()

In [58]:
texts = ["Hey how are you doing today?", "The weather was great in San Francisco yesterday.", "Hey how are you doing today?"]
embeddings_list = []
for text in texts:
    embeddings_list.append(get_embeddings(text))
for embeddings in embeddings_list:
    print(embeddings.size())

tensor([[[ 1.6484,  2.4219,  0.7773,  ..., -1.6953,  1.7656,  2.8750],
         [-0.2119, -0.2891, -1.9609,  ..., -1.4453, -2.4375, -1.3359],
         [-2.5625,  1.8984,  3.0000,  ...,  2.3281,  0.5820,  0.9492],
         ...,
         [-1.0547, -0.5547,  0.2383,  ...,  1.3594, -4.3125, -0.1562],
         [-1.8047, -1.1719,  2.4219,  ...,  1.3828, -4.2812,  0.2930],
         [-1.3906, -2.2344,  0.4395,  ...,  1.3047, -1.1797,  2.2969]]])
torch.Size([1, 8, 4096])
tensor([[[ 1.6484,  2.4219,  0.7773,  ..., -1.6953,  1.7656,  2.8750],
         [ 0.0549, -0.0649, -4.6250,  ..., -1.3516, -0.0669,  2.3438],
         [ 0.1650, -3.3125,  6.2188,  ..., -1.4531, -0.6914, -1.3281],
         ...,
         [-2.1250,  1.9219,  0.8242,  ...,  2.1406, -2.9062, -0.0486],
         [-0.8789,  0.2949,  0.7773,  ...,  1.9609, -3.5938,  1.0391],
         [-1.9375, -1.4844, -2.7656,  ...,  0.7773,  1.1016,  1.0000]]])
torch.Size([1, 10, 4096])
tensor([[[ 1.6484,  2.4219,  0.7773,  ..., -1.6953,  1.7656,  2.8

In [43]:
print(f"gpu mem used {torch.cuda.max_memory_allocated(device=None)} memory")

gpu mem used 31081201152 memory


In [44]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
# # Calculate cosine similarity
similarity = cosine_similarity(embeddings_list[0], embeddings_list[0])
print(similarity)

ValueError: Expected 2D array, got 1D array instead:
array=[-1.4532471  -0.36523438  0.8088379  ...  0.6694336  -2.0961914
  0.4177246 ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.